In [13]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate


chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What do you know about {country}?"),
        ("ai", "{answer}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a geography expert, you give short answer.",
        ),
        example_prompt,
        ("human", "What do you know about {country}?"),
    ]
)

chain = final_prompt | chat

In [13]:
# This is FewShotPromptTemplate (not FewShotChatMessagePromptTemplate)
chain.invoke({"country": "Korea"})

AI: 
        I know this:
        Capital: Seoul
        Language: Korean
        Food: Kimchi and Bibimbap
        Currency: South Korean won

AIMessageChunk(content='AI: \n        I know this:\n        Capital: Seoul\n        Language: Korean\n        Food: Kimchi and Bibimbap\n        Currency: South Korean won')

In [5]:
# gpt-4o-mini-2024-07-18
chain.invoke({"country": "Korea"})

Korea is divided into two distinct countries: North Korea (officially the Democratic People's Republic of Korea) and South Korea (officially the Republic of Korea). Here are some key points about each:

### North Korea:
- **Capital**: Pyongyang
- **Government**: Single-party state led by the Workers' Party of Korea, with a strong emphasis on a centralized government and military-first policy.
- **Language**: Korean
- **Economy**: Primarily state-controlled, with limited market reforms. The economy faces significant challenges, including international sanctions and food shortages.
- **Culture**: Heavily influenced by the regime, with a focus on propaganda and the cult of personality surrounding its leaders.

### South Korea:
- **Capital**: Seoul
- **Government**: Democratic republic with a multi-party system.
- **Language**: Korean
- **Economy**: One of the largest economies in Asia, known for its advanced technology, manufacturing (especially electronics and automobiles), and a vibrant

AIMessageChunk(content="Korea is divided into two distinct countries: North Korea (officially the Democratic People's Republic of Korea) and South Korea (officially the Republic of Korea). Here are some key points about each:\n\n### North Korea:\n- **Capital**: Pyongyang\n- **Government**: Single-party state led by the Workers' Party of Korea, with a strong emphasis on a centralized government and military-first policy.\n- **Language**: Korean\n- **Economy**: Primarily state-controlled, with limited market reforms. The economy faces significant challenges, including international sanctions and food shortages.\n- **Culture**: Heavily influenced by the regime, with a focus on propaganda and the cult of personality surrounding its leaders.\n\n### South Korea:\n- **Capital**: Seoul\n- **Government**: Democratic republic with a multi-party system.\n- **Language**: Korean\n- **Economy**: One of the largest economies in Asia, known for its advanced technology, manufacturing (especially electr

In [10]:
# gpt-4o-mini-2024-07-18
chain.invoke({"country": "Germany"})

Here are some key facts about Germany:

- **Capital**: Berlin
- **Language**: German
- **Currency**: Euro (€)
- **Geography**: Germany is located in Central Europe and is bordered by nine countries, including France, Poland, and the Netherlands. It has a diverse landscape that includes the Alps in the south, the North Sea and Baltic Sea coasts, and numerous rivers, including the Rhine and Danube.
- **Economy**: Germany has the largest economy in Europe and is known for its engineering, automotive industry, and manufacturing. Major companies include Volkswagen, BMW, and Siemens.
- **Culture**: Germany has a rich cultural heritage, known for contributions to philosophy, music, literature, and art. Notable figures include composers like Beethoven and Bach, and philosophers like Kant and Nietzsche.
- **History**: Germany has a complex history, including the Holy Roman Empire, the rise and fall of the Nazi regime, and the division of East and West Germany during the Cold War. The country re

AIMessageChunk(content='Here are some key facts about Germany:\n\n- **Capital**: Berlin\n- **Language**: German\n- **Currency**: Euro (€)\n- **Geography**: Germany is located in Central Europe and is bordered by nine countries, including France, Poland, and the Netherlands. It has a diverse landscape that includes the Alps in the south, the North Sea and Baltic Sea coasts, and numerous rivers, including the Rhine and Danube.\n- **Economy**: Germany has the largest economy in Europe and is known for its engineering, automotive industry, and manufacturing. Major companies include Volkswagen, BMW, and Siemens.\n- **Culture**: Germany has a rich cultural heritage, known for contributions to philosophy, music, literature, and art. Notable figures include composers like Beethoven and Bach, and philosophers like Kant and Nietzsche.\n- **History**: Germany has a complex history, including the Holy Roman Empire, the rise and fall of the Nazi regime, and the division of East and West Germany dur

In [8]:
chat = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)
chain = final_prompt | chat
chain.invoke({"country": "Germany"})


        I know this:
        Capital: Berlin
        Language: German
        Food: Bratwurst and Sauerkraut
        Currency: Euro
        

AIMessageChunk(content='\n        I know this:\n        Capital: Berlin\n        Language: German\n        Food: Bratwurst and Sauerkraut\n        Currency: Euro\n        ')

In [12]:
# gpt-4o-mini-2024-07-18
# add "you give short answer."
chain.invoke({"country": "Germany"})


        I know this:
        Capital: Berlin
        Language: German
        Food: Sausages and Sauerkraut
        Currency: Euro
        

AIMessageChunk(content='\n        I know this:\n        Capital: Berlin\n        Language: German\n        Food: Sausages and Sauerkraut\n        Currency: Euro\n        ')